In [259]:
import os
import pandas as pd
import numpy as np
data_path = "./data/" 
dic = {}

cols = ["Div","Date","HomeTeam","AwayTeam",
       "FTHG","FTAG","FTR","HTHG","HTAG","HTR"
]

matches = pd.DataFrame(None,columns=cols)

for f in [x for x in os.listdir(data_path)]: #if "prem" in x]:
    print(f)
    results_df = pd.read_csv(data_path+f,usecols=cols,parse_dates=True)
    
    results_df = results_df.dropna(how="all") # kill empty last rows, below tests this isn't deleting any matches
    
    if f.split("_")[0] == "prem": # premiership, 20 teams play each other twice, 20*19 total matches
        assert len(results_df) == 20*19
    else:
        assert len(results_df) == 24*23 # lower leagues, 24 teams play 2x, 24*23 total
    
    results_df["Season"] = (f.split("_")[-1].split(".")[0])
    results_df["Season"] = results_df["Season"]
    
    matches = pd.concat([results_df,matches])

matches["Date"] = pd.to_datetime(matches["Date"],dayfirst=True)
matches = matches.sort_values(by="Date")
matches = matches.reset_index(drop=True)

prem_1314.csv
league_2_1213.csv
champ_1314.csv
prem_1617.csv
league_1_1213.csv
league_2_1516.csv
league_2_1415.csv
league_1_1415.csv
prem_1213.csv
league_2_1617.csv
champ_1213.csv
champ_1516.csv
league_1_1617.csv
champ_1415.csv
champ_1617.csv
league_1_1314.csv
league_1_1516.csv
league_2_1314.csv
prem_1415.csv
prem_1516.csv


In [303]:
#setup ELOs, scale for league
starting_league = matches.loc[matches["Season"] == matches["Season"].min()] # Earliest season
# teams = {team : 1000 for team in matches["HomeTeam"].unique()}
teams = {}
for team in matches["HomeTeam"].unique():
    if team not in list(teams.keys()):
        try:
            team_starting_league = starting_league.loc[starting_league["HomeTeam"] == team]["Div"].mode()[0]
#             print(team_starting_league)
            if team_starting_league == "E3":
                teams[team] = -1000
            elif team_starting_league == "E2":
                teams[team] = 0
            elif team_starting_league == "E1":
                teams[team] = 1000
            elif team_starting_league == "E0":
                teams[team] = 2000
        except: #must be lower than conference, give 0
            teams[team] = -2000
    
k=32 # k_factor, scales ELO change, 32 = chess k_factor

for index,match in matches.iterrows():
#     print(match)
#     print(match["AwayTeam"])
    home_team = match["HomeTeam"]
    away_team = match["AwayTeam"]
    rating_home = teams[home_team]
    rating_away = teams[away_team]
    
    q_home = 10**(rating_home/400)
    q_away = 10**(rating_away/400)

    e_home =  q_home/(q_home + q_away)# expected value home wins
    e_away =  q_away/(q_home + q_away) 
    
    if match["FTR"] == "H": # Home wins
        s_home = 1
        s_away = 0
        
    elif match["FTR"] == "D": # Draw
        s_home = 0.5
        s_away = 0.5
    
    elif match["FTR"] == "A": # Draw
        s_home = 0
        s_away = 1
    
    teams[home_team] = rating_home + k*(s_home - e_home)
    teams[away_team] = rating_away + k*(s_away - e_away)

In [304]:
print(pd.DataFrame(teams,index=range(1)).T.sort_values(by=0,ascending=False))

                          0
Chelsea         1979.771741
Tottenham       1967.992200
Man City        1914.837061
Arsenal         1891.157133
Liverpool       1881.600947
Man United      1867.374993
Everton         1771.393904
Leicester       1725.687428
Southampton     1718.803001
West Ham        1712.240728
Swansea         1692.263495
Stoke           1687.863719
Crystal Palace  1656.670132
West Brom       1654.015753
Bournemouth     1644.917840
Watford         1573.248527
Sunderland      1546.170137
Burnley         1513.732999
Hull            1444.075708
Middlesbrough   1371.029637
Newcastle       1346.204310
Norwich         1123.715629
Brighton        1094.253351
Aston Villa     1073.527916
Fulham          1060.716855
Sheffield Weds  1042.977227
Reading         1024.295162
Leeds            981.883677
Derby            971.476959
Huddersfield     966.304155
...                     ...
Northampton     -311.460605
Blackpool       -469.887964
Doncaster       -547.007200
Colchester      -608